In [49]:
import torch as tr
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

In [50]:
device = tr.device('cuda' if tr.cuda.is_available() else 'cpu')
# print(device)

In [51]:
# #Force Field

# X,Y = np.linspace(-0.75,0.75,43),np.linspace(-0.75,0.75,43)

# # def U(x,y,U_0=0.4): #potential function, U_0 is the amplitude, x and y are the coordinates, Gradient needs to be calculated manually in forward pass.
# #     if (x**2+y**2)**0.5<=0.5:
# #         return tr.tensor((16*U_0*((x**2+y**2)**0.5-0.25)**2),dtype=tr.half)
# #     else:
# #         return tr.tensor(0,dtype=tr.half)

# def U(x,y,U_0=0.4): #potential function, U_0 is the amplitude, x and y are the coordinates, Gradient needs to be calculated manually in forward pass.
#     if (x**2+y**2)**0.5<=0.5:
#         return 16*U_0*((x**2+y**2)**0.5-0.25)**2
#     else:
#         return 0
# # Z = U(X,Y)
# # F = tr.from_numpy(np.gradient(Z))
# # print(F.shape)
dr = tr.tensor([1,0],dtype=tr.float)
dr *=2
print(dr)

tensor([2., 0.])


In [52]:
# #Toy Force Field
# x = np.linspace(-0.75, 0.75, 100)
# y = np.linspace(-0.75, 0.75, 100)
# X, Y = np.meshgrid(x, y)

# # Calculate the force field
# force_field_x = -np.abs(Y)
# force_field_y = np.zeros_like(X)

# # Create the meshgrid for force field
# F = tr.tensor(tr.from_numpy(np.stack((force_field_x, force_field_y), axis=2)),dtype=tr.float)

t1=tr.tensor(1,dtype=tr.float)
t2=tr.tensor(1,dtype=tr.float)
t1-=t2
print(t1)
T = tr.stack([t1,t2])
print(T)

tensor(0.)
tensor([0., 1.])


In [53]:
#Agent Class
class Agent(nn.Module):
    def __init__(self, x=tr.tensor(-0.5, dtype=tr.float), y=tr.tensor(0,dtype=tr.float), b=tr.tensor(0,dtype=tr.float)):#initialize agent at location (-0.5,-0.5)
        super(Agent, self).__init__()
        self.x=x #x coordinate
        self.y=y #y coordinate
        self.b=b #bias
        self.input = nn.Linear(5, 64,dtype=tr.float) #input layer
        self.hidden1 = nn.Linear(64, 64,dtype=tr.float) #hidden layer
        self.hidden2 = nn.Linear(64, 32, dtype = tr.float) #hidden layer
        self.output = nn.Linear(32, 4,dtype=tr.float) 
        self.activation = nn.ReLU() #activation function

    def forward(self, x, network = None): #forward pass
        if network is None: network = self
        x1 = self.activation(self.input(x))
        x2 = self.activation(self.hidden1(x1))
        x3 = self.activation(self.hidden2(x2))
        x4 = self.activation(self.output(x3))
        return x4
    

    def move(self,Q,e):
        if tr.rand(size=(1,))<e:
            L = tr.randint(0, 4, (1,))
        else: L = tr.argmax(Q)
        
        dx,dy = tr.tensor(0.,dtype=tr.float),tr.tensor(0.,dtype=tr.float)
        a = tr.tensor(0.0357,dtype=tr.float)
        if L.item()==0: dy = a #up
        elif L.item()==1: dy = -a #down
        elif L.item()==2: dx = a #right
        elif L.item()==3: dx = -a #left
        else: raise Exception('Invalid action')
        if (-0.75<=self.x<=0.75).item() == False: raise Exception('Invalid x coordinate')
        if (-0.75<=self.y<=0.75).item() == False: raise Exception('Invalid y coordinate')
        
        if (-0.75<=self.x+dx<=0.75).item() == False:
            dx=0
        if (-0.75<=self.y+dy<=0.75).item() == False:
            dy=0
        self.x+=dx
        self.y+=dy
        #if dx == dy == 0: print('stuck')

def R(x,y,F,L):#Reward function, F is the force field, L is the action
    u = tr.tensor([-1,0],dtype=tr.float).to(device) #normalized Force Field
    dx,dy = 0,0
    if L.item()==0 or L.item()==1: dy = 1
    elif L.item()==2 or L.item()==3: dx = 1
    else: raise Exception('Invalid action')

    dr = tr.tensor([dx,dy],dtype=tr.float).to(device)
    e = dr - u/tr.norm(dr-u,p=2)
    dr*=0.0357
    dt = 0.0357/tr.dot((e+F),dr).item()
    r = -0.5*dt/1000-0.5*tr.norm(tr.tensor([x-0.5,y]),p=2)
    if r.shape != tr.Size([]): raise Exception('Reward is not a scalar')
    return r

# def R(x,y):
#     return -tr.norm(tr.tensor([x-0.5,y]),p=2)




In [54]:
#Simulation
agent = Agent().to(device)

t_agent = Agent().to(device)
t_agent.load_state_dict(agent.state_dict())

t_agent.eval()

t_update = 100
optimizer = tr.optim.Adam(agent.parameters(), lr=0.001)
dT = 0
g = 0.8
max_steps = 2000
X,Y=[],[]
print(agent.x,agent.y)


while dT<max_steps:
    
    x0,y0 = agent.x.numpy(),agent.y.numpy()
    X.append(agent.x.numpy())
    Y.append(agent.y.numpy())
    dT+=1
    if dT%t_update==0:#update target agent every 100 steps
        t_agent.load_state_dict(agent.state_dict())
        print(agent.x,agent.y)


    s = tr.stack([agent.x, agent.y, -tr.abs(agent.y), tr.tensor(0,dtype=tr.float), agent.b]).to(device) #state
    Q1 = agent.forward(s) #Q value
    e = 1-dT/max_steps #epsilon
    agent.move(Q1,tr.tensor(e,dtype=tr.float))      	    #move agent
    x1,y1 = agent.x.numpy(),agent.y.numpy()
    if np.abs(x0-x1)>0.0357 or np.abs(y1-y0)>0.0357: raise Exception('Agent moved too far')


    s2 = tr.stack([agent.x, agent.y, -tr.abs(agent.y), tr.tensor(0,dtype=tr.float), agent.b]).to(device) #state
    Q2 = agent.forward(s2,network=t_agent) #Q value
    
    F = tr.tensor([-agent.y,0],dtype=tr.float).to(device)
    tQ = R(agent.x.to(device), agent.y.to(device), F, tr.argmax(Q1))+ g*tr.max(Q2)         #target Q value
    criterion = nn.MSELoss().to(device)     #1/2*(tQ-Q1)**2 #loss
    loss = criterion(tQ,tr.max(Q1))         #loss


    optimizer.zero_grad()                   #zero gradients
    loss.backward()                         #backpropagate
    optimizer.step()                        #update weights
    if (tr.round(agent.x,decimals = 2),tr.round(agent.y,decimals = 2)) == (0.5000,0.0000):
        print('success')
        break
print(agent.x,agent.y)
# print(tr.round(agent.x,decimals = 1),tr.round(agent.y,decimals = 1))


tensor(-0.5000) tensor(0.)
tensor(-0.3929) tensor(0.4998)
tensor(-0.3572) tensor(0.7497)
tensor(-0.7142) tensor(0.6783)
tensor(-0.4643) tensor(0.7497)
tensor(-0.5357) tensor(0.6426)
tensor(-0.7142) tensor(0.7497)
tensor(-0.4643) tensor(0.6783)
tensor(-0.6785) tensor(0.7497)
tensor(-0.4643) tensor(0.7140)
tensor(-0.3215) tensor(0.7497)
tensor(-0.1430) tensor(0.7497)
tensor(-0.0716) tensor(0.7497)
tensor(-0.2144) tensor(0.7497)
tensor(-0.1430) tensor(0.7497)
tensor(-0.1073) tensor(0.7497)
tensor(-0.3929) tensor(0.7497)
tensor(-0.5714) tensor(0.7497)
tensor(-0.5357) tensor(0.7497)
tensor(-0.5714) tensor(0.7497)
tensor(-0.6071) tensor(0.7497)
tensor(-0.6071) tensor(0.7497)
